In [43]:
import quandl, math
import pandas as pd
import numpy as np
from sklearn import preprocessing, model_selection, svm
from sklearn.linear_model import LinearRegression

In [2]:
df = quandl.get('WIKI/GOOGL')

In [3]:
df.head()

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
2004-08-19,100.01,104.06,95.96,100.335,44659000.0,0.0,1.0,50.159839,52.191109,48.128568,50.322842,44659000.0
2004-08-20,101.01,109.08,100.50,108.310,22834300.0,0.0,1.0,50.661387,54.708881,50.405597,54.322689,22834300.0
2004-08-23,110.76,113.48,109.05,109.400,18256100.0,0.0,1.0,55.551482,56.915693,54.693835,54.869377,18256100.0
2004-08-24,111.24,111.60,103.57,104.870,15247300.0,0.0,1.0,55.792225,55.972783,51.945350,52.597363,15247300.0
2004-08-25,104.76,108.00,103.88,106.000,9188600.0,0.0,1.0,52.542193,54.167209,52.100830,53.164113,9188600.0


In [4]:
df.shape

(3424, 12)

In [5]:
df =df[['Adj. Open','Adj. High','Adj. Low','Adj. Close','Adj. Volume']]

In [6]:
df.tail()

,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,
2018-03-21,1092.57,1108.70,1087.21,1094.00,1990515.0
2018-03-22,1080.01,1083.92,1049.64,1053.15,3418154.0
2018-03-23,1051.37,1066.78,1024.87,1026.55,2413517.0
2018-03-26,1050.60,1059.27,1010.58,1054.09,3272409.0
2018-03-27,1063.90,1064.54,997.62,1006.94,2940957.0


In [7]:
#percentage = (new-old)/old

#high low percentage
df['HL_PCT'] = (df['Adj. High'] - df['Adj. Close']) / df['Adj. Close'] * 100.0

#percentage change
df['PCT_Change'] = (df['Adj. Close'] - df['Adj. Open']) / df['Adj. Open'] * 100.0


In [8]:
df = df[['Adj. Close','HL_PCT','PCT_Change','Adj. Volume',]]

In [9]:
df.head()

,Adj. Close,HL_PCT,PCT_Change,Adj. Volume
Date,,,,
2004-08-19,50.322842,3.712563,0.324968,44659000.0
2004-08-20,54.322689,0.710922,7.227007,22834300.0
2004-08-23,54.869377,3.729433,-1.227880,18256100.0
2004-08-24,52.597363,6.417469,-5.726357,15247300.0
2004-08-25,53.164113,1.886792,1.183658,9188600.0


In [10]:
forecast_col = 'Adj. Close'

In [12]:
df.fillna(-99999, inplace=True)

In [81]:
forecast_out = int(math.ceil(0.01*len(df)))   #100%

In [82]:
forecast_out

25

In [83]:
len(df)

2439

In [84]:
df['label'] = df[forecast_col].shift(-forecast_out)

In [85]:
df.head()

,Adj. Close,HL_PCT,PCT_Change,Adj. Volume,label
Date,,,,,
2004-08-19,50.322842,3.712563,0.324968,44659000.0,60.100525
2004-08-20,54.322689,0.710922,7.227007,22834300.0,59.313094
2004-08-23,54.869377,3.729433,-1.227880,18256100.0,63.626409
2004-08-24,52.597363,6.417469,-5.726357,15247300.0,65.742942
2004-08-25,53.164113,1.886792,1.183658,9188600.0,65.000651


In [86]:
df.dropna(inplace=True)

In [87]:
df.head()

,Adj. Close,HL_PCT,PCT_Change,Adj. Volume,label
Date,,,,,
2004-08-19,50.322842,3.712563,0.324968,44659000.0,60.100525
2004-08-20,54.322689,0.710922,7.227007,22834300.0,59.313094
2004-08-23,54.869377,3.729433,-1.227880,18256100.0,63.626409
2004-08-24,52.597363,6.417469,-5.726357,15247300.0,65.742942
2004-08-25,53.164113,1.886792,1.183658,9188600.0,65.000651


In [88]:
#features
X= np.array(df.drop(['label'],1))
X.shape

(2414, 4)

In [89]:
X[4:]

array([[ 5.31641125e+01,  1.88679245e+00,  1.18365788e+00,
         9.18860000e+06],
       [ 5.41220696e+01,  3.70679270e-02,  2.82039066e+00,
         7.09480000e+06],
       [ 5.32393448e+01,  2.32689590e+00, -1.80388529e+00,
         6.21170000e+06],
       ...,
       [ 6.01481716e+02,  1.06233062e+00, -1.03564945e+00,
         3.22250000e+06],
       [ 6.00433481e+02,  1.03996124e+00, -2.32509688e-01,
         3.36600000e+06],
       [ 5.93351620e+02,  2.24759940e+00, -1.92902322e+00,
         6.41050000e+06]])

In [90]:
X= preprocessing.scale(X)
X[4:]

array([[-2.01320046,  0.58796301,  0.73293079, -0.12388243],
       [-2.00411462, -0.92512438,  1.73020551, -0.36450439],
       [-2.01248691,  0.94797066, -1.08740379, -0.46599129],
       ...,
       [ 3.18736846, -0.08645244, -0.61931138, -0.80951367],
       [ 3.17742637, -0.10475075, -0.12995174, -0.79302248],
       [ 3.1102578 ,  0.88310558, -1.16365136, -0.44314496]])

In [91]:
#labels
y=np.array(df['label'])
y.shape

(2414,)

In [92]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X,y, test_size=0.2)

In [93]:
print(X_train.shape,y_train.shape)
print(X_test.shape, y_test.shape)

(1931, 4) (1931,)
(483, 4) (483,)


In [126]:
##Linear Regression
clf = LinearRegression(n_jobs=-1)
clf.fit(X_train, y_train)
accuracy = clf.score(X_test,y_test)
accuracy

0.9385717961757716

In [127]:
help(LinearRegression)

Help on class LinearRegression in module sklearn.linear_model.base:

class LinearRegression(LinearModel, sklearn.base.RegressorMixin)
 |  LinearRegression(fit_intercept=True, normalize=False, copy_X=True, n_jobs=None)
 |  
 |  Ordinary least squares Linear Regression.
 |  
 |  Parameters
 |  ----------
 |  fit_intercept : boolean, optional, default True
 |      whether to calculate the intercept for this model. If set
 |      to False, no intercept will be used in calculations
 |      (e.g. data is expected to be already centered).
 |  
 |  normalize : boolean, optional, default False
 |      This parameter is ignored when ``fit_intercept`` is set to False.
 |      If True, the regressors X will be normalized before regression by
 |      subtracting the mean and dividing by the l2-norm.
 |      If you wish to standardize, please use
 |      :class:`sklearn.preprocessing.StandardScaler` before calling ``fit`` on
 |      an estimator with ``normalize=False``.
 |  
 |  copy_X : boolean, o

In [123]:
##Support Vector Machine - Regression
clf = svm.SVR(gamma='scale',kernel='poly')
clf.fit(X_train, y_train )
accuracy = clf.score(X_test, y_test)
accuracy

0.9382111348937929

In [128]:
help(svm.SVR)

Help on class SVR in module sklearn.svm.classes:

class SVR(sklearn.svm.base.BaseLibSVM, sklearn.base.RegressorMixin)
 |  SVR(kernel='rbf', degree=3, gamma='auto_deprecated', coef0=0.0, tol=0.001, C=1.0, epsilon=0.1, shrinking=True, cache_size=200, verbose=False, max_iter=-1)
 |  
 |  Epsilon-Support Vector Regression.
 |  
 |  The free parameters in the model are C and epsilon.
 |  
 |  The implementation is based on libsvm.
 |  
 |  Read more in the :ref:`User Guide <svm_regression>`.
 |  
 |  Parameters
 |  ----------
 |  kernel : string, optional (default='rbf')
 |       Specifies the kernel type to be used in the algorithm.
 |       It must be one of 'linear', 'poly', 'rbf', 'sigmoid', 'precomputed' or
 |       a callable.
 |       If none is given, 'rbf' will be used. If a callable is given it is
 |       used to precompute the kernel matrix.
 |  
 |  degree : int, optional (default=3)
 |      Degree of the polynomial kernel function ('poly').
 |      Ignored by all other kernels

In [132]:
df.iloc[3].name

Timestamp('2004-08-24 00:00:00')

In [134]:
help(df.iloc)

Help on _iLocIndexer in module pandas.core.indexing object:

class _iLocIndexer(_LocationIndexer)
 |  Purely integer-location based indexing for selection by position.
 |  
 |  ``.iloc[]`` is primarily integer position based (from ``0`` to
 |  ``length-1`` of the axis), but may also be used with a boolean
 |  array.
 |  
 |  Allowed inputs are:
 |  
 |  - An integer, e.g. ``5``.
 |  - A list or array of integers, e.g. ``[4, 3, 0]``.
 |  - A slice object with ints, e.g. ``1:7``.
 |  - A boolean array.
 |  - A ``callable`` function with one argument (the calling Series, DataFrame
 |    or Panel) and that returns valid output for indexing (one of the above).
 |    This is useful in method chains, when you don't have a reference to the
 |    calling object, but would like to base your selection on some value.
 |  
 |  ``.iloc`` will raise ``IndexError`` if a requested indexer is
 |  out-of-bounds, except *slice* indexers which allow out-of-bounds
 |  indexing (this conforms with python/num